In [2]:
from meshparty import trimesh_io, trimesh_vtk

from dotenv import load_dotenv
import os
import numpy as np

load_dotenv(dotenv_path=os.path.expanduser('~/.config/pinky100/.env'))
dataset_name = os.getenv('DATASET_NAME')
sql_database_uri_base = os.getenv('MATERIALIZATION_DATABASE_URI_BASE')
mesh_cv_path = os.getenv('GRAPHENE_SOURCE')
voxel_resolution = np.array(os.getenv('VOXEL_RESOLUTION').split(',')).astype(float)
voxel_scaling = np.array(os.getenv('VOXEL_SCALING').split(',')).astype(float)

In [3]:
mm = trimesh_io.MeshMeta(dataset_name='pinky100', voxel_scaling=voxel_scaling)

In [4]:
mesh = mm.mesh(seg_id = 648518346349537426)

Downloading: 100%|█████████▉| 2698/2701 [01:12<00:00, 39.01it/s]
Downloading: 5400it [01:12, 74.29it/s]                          

Decoding Mesh Buffer: 100%|██████████| 2701/2701 [00:00<00:00, 14449.58it/s]


In [5]:
from meshparty import trimesh_vtk
ma = trimesh_vtk.mesh_actor(mesh, opacity=0.5, color=(0.5, 0.5, 0.5))
# trimesh_vtk.render_actors([ma])

In [ ]:
ind = np.array([40300])
pca = trimesh_vtk.point_cloud_actor(mesh.vertices[ind], size=1000, opacity=0.4, color=(1,0,0))
trimesh_vtk.render_actors([ma, pca])

In [ ]:
import trimesh.ray
from trimesh.ray import ray_pyembree

ray_inter = ray_pyembree.RayMeshIntersector(mesh)

In [ ]:
starts = (mesh.vertices-mesh.vertex_normals)[ind,:]

In [ ]:
vs = -mesh.vertex_normals[ind,:]

In [ ]:
rtrace = ray_inter.intersects_location(starts, vs, multiple_hits=False)

In [ ]:
np.linalg.norm(mesh.vertices[ind[rtrace[1]]] - rtrace[0], axis=1)

In [ ]:
mesh.face_normals[rtrace[2]]

In [ ]:
vs[0,:]

In [ ]:
costh = np.dot( vs[0,:], mesh.face_normals[rtrace[2],:].T )

In [ ]:
weight = 1/np.arccos(costh)

In [ ]:
weight

In [ ]:
 mesh.face_normals[rtrace[2],:]

In [ ]:
from scipy.constants import golden_ratio

In [ ]:
golden_ratio

In [107]:
from trimesh.ray import ray_pyembree
from scipy.linalg import block_diag

def vogel_disk_sampler(num_points, radius=1):

    golden_angle = np.pi * (3 - np.sqrt(5))
    thetas = golden_angle * np.arange(num_points)
    radii = radius * np.sqrt(np.arange(num_points)/num_points)

    xs = radii * np.cos(thetas)
    ys = radii * np.sin(thetas)
    
    return xs, ys

def unit_vector_sampler(num_points, widest_angle=np.pi/3):
    if np.abs(widest_angle) > np.pi:
        print('')
        return None
    xs, ys = vogel_disk_sampler(num_points, radius=1)
    zs = 1/np.tan(widest_angle) * np.ones(num_points)
    Vs = np.vstack((xs, ys, zs)).T
    return Vs / np.linalg.norm(Vs, axis=1)[:, np.newaxis]

def Rx( ang ):
    return np.array([[1, 0, 0],
                     [0, np.cos(ang), -np.sin(ang)],
                     [0, np.sin(ang), np.cos(ang)]])

def Ry( ang ):
    return np.array([[np.cos(ang), 0, np.sin(ang)],
                     [0,           1,       0    ],
                     [-np.sin(ang),0, np.cos(ang)]])

def Rz( ang ):
    return np.array([[np.cos(ang), -np.sin(ang), 0],
                     [np.sin(ang), np.cos(ang), 0],
                     [0,  0, 1]])

def vec_actor(vec, origin=np.array([0,0,0]), color=(0,0,0), linewidth=2, opacity=1):
    if type(vec) is list:
        vec=np.array(vec)
    return trimesh_vtk.linked_point_actor(origin.reshape(1,3),
                                              origin+vec.reshape(1,3), line_width=linewidth,
                                              color=color, opacity=opacity)

import multiwrapper.multiprocessing_utils as mu

def _rotated_cone(data):
    phi, theta, vs_raw = data
    Rtrans = np.dot(Rz(phi), Ry(theta))
    return np.dot(Rtrans, vs_raw.T).T
    
def oriented_vector_cones(center_vectors, num_points, widest_angle=np.pi/3, normalize=False):
    
    if normalize:
        cv_norm = center_vectors / np.linalg.norm(center_vector, axis=1)[:, np.newaxis]
    else:
        cv_norm = center_vectors
        
    thetas = np.arccos(cv_norm[:, 2])
    phis = np.arctan2(cv_norm[:, 1], cv_norm[:, 0])
    
    vs_raw = unit_vector_sampler(num_points, widest_angle=widest_angle)
    
    Rtranses = []
    data = []
    for phi, theta in zip(phis, thetas):
        data.append((phi, theta, vs_raw))
    vector_cones = mu.multiprocess_func(_rotated_cone, data)
    return vector_cones

def _multi_angle_weighted_distance(data):
    ds, angles, weights = data
    return angle_weighted_distance(ds, angles, weights)

def angle_weighted_distance(ds, angles, weights):
    if len(ds)==0 or np.nansum(weights)==0:
        return np.nan
    
    med_angle = np.median(angles)
    std_angle = np.std(angles)
    min_angle = med_angle-std_angle
    max_angle = max(med_angle+std_angle, np.pi / 2)
    good_rows = np.logical_and(angles>=min_angle, angles<=max_angle)
    
    nanaverage = np.nansum(ds[good_rows] * weights[good_rows]) / np.nansum(weights[good_rows])
    return nanaverage

def all_angle_weighted_distances(ds, angles, weights, rep_inds, inds):

    data = []
    real_inds, slice_bnds = np.unique(rep_inds, return_index=True)
    
    ind_map = []
    for ii in range(len(real_inds)-1):
        row = slice(slice_bnds[ii], slice_bnds[ii+1])
        data.append((ds[row], angles[row], weights[row]))
        ind_map.append(real_inds[ii])
    row = slice(slice_bnds[-1], len(ds))
    data.append( (ds[row], angles[row], weights[row]) )
    ind_map.append(real_inds[-1])
        
    rs = mu.multiprocess_func(_multi_angle_weighted_distance, data)
    
    rs_out = np.nan * np.zeros(len(inds))
    rs_out[np.array(ind_map)] = rs
    
    return rs_out

def _compute_ray_vectors(mesh, mesh_inds, num_points, cone_angle):
    return np.vstack( oriented_vector_cones(-mesh.vertex_normals[mesh_inds], num_points, cone_angle) )

def shape_diameter_function(mesh, mesh_inds, num_points=30, cone_angle=np.pi/3):

    start = (mesh.vertices-mesh.vertex_normals)[mesh_inds,:]
    rep_inds = np.concatenate([ii*np.ones(num_points, dtype=int) for ii in range(start.shape[0])])
    starts = start[rep_inds]
    
    vs = _compute_ray_vectors(mesh, mesh_inds, num_points, cone_angle)
    
    ray_inter = ray_pyembree.RayMeshIntersector(mesh)
    rtrace = ray_inter.intersects_location(starts, vs, multiple_hits=False)
    
    hit_rows = rtrace[1]
    ds = np.linalg.norm(rtrace[0] - starts[hit_rows], axis=1)
    angles = np.arccos(np.sum(mesh.face_normals[rtrace[2]] * vs[hit_rows], axis=1))
    weights = 1/angles
    good_rows = np.isfinite(weights)
    
    rs = all_angle_weighted_distances(ds[good_rows], angles[good_rows], weights[good_rows], rep_inds[hit_rows[good_rows]], mesh_inds)
    return rs

In [14]:
vs_raw = unit_vector_sampler(30, widest_angle=np.pi/3)

In [207]:
from numpy import matlib

array([0.        , 0.        , 1.        , ..., 0.76331346, 0.40117616,
       0.50636968])

In [15]:
vs_long = np.concatenate( len(vinds)*[vs_raw] ).ravel().T

In [27]:
import time

In [28]:
t0 = time.time()
vs_rot = np.dot(test.astype(np.float32), np.vstack(len(vinds) * [vs_raw.T]).astype(np.float32))
t1 = time.time()
print(t1-t0)

67.22831201553345


In [29]:
t0 = time.time()
vs_rot = np.dot(test, np.vstack(len(vinds) * [vs_raw.T]))
t1 = time.time()
print(t1-t0)

37.25190019607544


In [37]:
vinds = np.random.permutation(np.arange(mesh.n_vertices))[0:20000]

In [38]:
t0 = time.time()
rs = shape_diameter_function(mesh, vinds, num_points=1)
t1 = time.time()
print(f'Duration: {t1-t0} seconds')

/Users/caseyschneider-mizell/anaconda3/envs/data_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: divide by zero encountered in true_divide


Duration: 14.736207246780396 seconds


In [39]:
t0 = time.time()
rs = shape_diameter_function(mesh, vinds, num_points=10)
t1 = time.time()
print(f'Duration: {t1-t0} seconds')

/Users/caseyschneider-mizell/anaconda3/envs/data_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: divide by zero encountered in true_divide


Duration: 14.358166217803955 seconds


In [40]:
t0 = time.time()
rs = shape_diameter_function(mesh, vinds, num_points=30)
t1 = time.time()
print(f'Duration: {t1-t0} seconds')

/Users/caseyschneider-mizell/anaconda3/envs/data_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: divide by zero encountered in true_divide


Duration: 16.190942764282227 seconds


In [41]:
np.nanmedian(rs)

431.70496749332085

In [ ]:
lpa = trimesh_vtk.linked_point_actor(vertices_a=np.zeros(vs.shape), vertices_b=vs, line_width=4)
trimesh_vtk.render_actors([lpa])

In [ ]:
base_vector = 2*np.random.rand(3)-1

bv_norm = base_vector / np.linalg.norm(base_vector)
theta = np.arccos(bv_norm[2])
phi=np.arctan2(bv_norm[1], bv_norm[0])

In [ ]:
phi

In [ ]:
phi=np.arctan2(bv_norm[1], bv_norm[0])

In [ ]:
Rtrans = np.dot(Rz(phi), Ry(theta))

vs = unit_vector_sampler(120, widest_angle=np.pi/6)

lpa = trimesh_vtk.linked_point_actor(vertices_a=np.zeros(vs.shape),
                                     vertices_b=vs, line_width=4)

rlpa = trimesh_vtk.linked_point_actor(vertices_a = np.zeros(vs.shape),
                                      vertices_b = np.dot(Rtrans, vs.T).T,
                                      color=(0, 0.8, 0.2),
                                      opacity=0.4,
                                      line_width = 4)

In [ ]:
num_pts = 300 

base_vector = 2*np.random.rand(3)-1
bv_norm = base_vector / np.linalg.norm(base_vector)

bva = vec_actor(2 * bv_norm, color=(0.8, 0.3, 0.3), linewidth=3)
lpa = trimesh_vtk.linked_point_actor(vertices_a=np.zeros((num_pts,3)),
                                     vertices_b=oriented_vector_cone(num_pts, base_vector, widest_angle=np.pi/6), line_width=4)

x_axis = vec_actor([1.3,0,0], linewidth=1, opacity=0.5, color=(0.5,0,0))
y_axis = vec_actor([0,1.3,0], linewidth=1, opacity=0.5, color=(0,0.5,0))
z_axis = vec_actor([0,0,1.3], linewidth=1, opacity=0.5, color=(0,0,0.5))

trimesh_vtk.render_actors([x_axis, y_axis, z_axis, bva, lpa])

In [52]:
ray_inter = ray_pyembree.RayMeshIntersector(mesh)

In [93]:
ind = 0
oriented_vector_cone(10, mesh.vertex_normals[ind]), mesh.vertex_normals[ind]

(array([[-0.94384392, -0.05384925, -0.32597381],
        [-0.92460163, -0.37777415,  0.04897466],
        [-0.76349825,  0.56745975, -0.30831776],
        [-0.51734291, -0.57659775, -0.63236962],
        [-0.88034947,  0.0786244 ,  0.46776385],
        [-0.40791969,  0.39315076, -0.8240352 ],
        [-0.58769057, -0.80908338,  0.00197069],
        [-0.70113031,  0.69160491,  0.17349045],
        [-0.23861778, -0.30229268, -0.92286548],
        [-0.72909119, -0.36806857,  0.57701955]]),
 array([-0.94384392, -0.05384925, -0.32597381]))

In [57]:
t0 = time.time()
rtrace = ray_inter.intersects_location(mesh.vertices-mesh.vertex_normals, -mesh.vertex_normals)
t1 = time.time()
print(t1-t0)

116.78793120384216


In [44]:
%load_ext line_profiler

In [47]:
vinds = np.random.permutation(np.arange(mesh.n_vertices))[0:100000]

In [52]:

cam = trimesh_vtk.render_actors([ma])

In [57]:
c=cam.GetActiveCamera()
pt=np.array(c.GetPosition())

array([253743.87988908, 189458.07841645,  56249.63229858])

In [56]:
from meshparty import mesh_filters

In [63]:
mf = mesh_filters.filter_spatial_distance_from_points(mesh,pts=[pt],d_max=40000)

In [67]:
mesh_filt = mesh.apply_mask(mf)
mesh_filt = mesh_filt.apply_mask( mesh_filters.filter_largest_component(mesh_filt) )

In [68]:
ma = trimesh_vtk.mesh_actor(mesh_filt)
trimesh_vtk.render_actors([ma])

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x116251a08

In [108]:
rs = shape_diameter_function(mesh_filt, np.arange(mesh_filt.n_vertices), num_points=30)

/Users/caseyschneider-mizell/anaconda3/envs/data_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in true_divide


In [109]:
rs = np.array(rs)
ma = trimesh_vtk.mesh_actor(mesh_filt, vertex_colors=rs/1000, opacity=1)
trimesh_vtk.render_actors([ma])

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x12a16d708

In [85]:
mesh_filt.write_to_file('test_dendrite.h5')

In [101]:
%lprun -f shape_diameter_function shape_diameter_function(mesh_filt, np.arange(mesh_filt.n_vertices), num_points=30)

/Users/caseyschneider-mizell/anaconda3/envs/data_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:120: RuntimeWarning: divide by zero encountered in true_divide


Timer unit: 1e-06 s

Total time: 49.894 s
File: <ipython-input-94-a30cc8e4fba1>
Function: shape_diameter_function at line 107

Line #      Hits         Time  Per Hit   % Time  Line Contents
   107                                           def shape_diameter_function(mesh, mesh_inds, num_points=30, cone_angle=np.pi/3):
   108                                           
   109         1       3769.0   3769.0      0.0      start = (mesh.vertices-mesh.vertex_normals)[mesh_inds,:]
   110         1     514113.0 514113.0      1.0      rep_inds = np.concatenate([ii*np.ones(num_points, dtype=int) for ii in range(start.shape[0])])
   111         1     203494.0 203494.0      0.4      starts = start[rep_inds]
   112                                               
   113         1    2594347.0 2594347.0      5.2      vs = _compute_ray_vectors(mesh, mesh_inds, num_points, cone_angle)
   114                                               
   115         1        442.0    442.0      0.0      ray_inter = 

0.0

In [ ]:
ind = np.array([40300])
pca = trimesh_vtk.point_cloud_actor(mesh.vertices[ind], size=100, opacity=0.4, color=(1,0,0))
# trimesh_vtk.render_actors([ma, pca])

In [ ]:
inds = np.array([40300, 185300])

In [ ]:
num_pts = 30
start = (mesh.vertices-mesh.vertex_normals)[inds,:]
rep_inds = np.concatenate([ii*np.ones(num_pts, dtype=int) for ii in range(start.shape[0])])
starts = start[rep_inds]

In [ ]:
wangle = np.pi/3
vs = np.vstack( [oriented_vector_cone(num_pts, -vertex_normal.reshape(3), wangle) for vertex_normal in mesh.vertex_normals[inds]])

In [ ]:
rtrace = ray_inter.intersects_location(starts, vs, multiple_hits=False)

hit_rows = rtrace[1]
ds = np.linalg.norm(rtrace[0] - starts[hit_rows], axis=1)
angles = np.arccos(np.sum(mesh.face_normals[rtrace[2]] * vs[hit_rows], axis=1))
weights = 1/angles

In [ ]:
all_angle_distances(ds, angles, weights, rep_inds, inds)

In [ ]:
np.array([angle_weighted_distance(data[data[:,3]== ii , 0:3]) for ii in range(len(inds))])

In [ ]:
out.statistic[1:].reshape(len(inds))

In [ ]:
out = stats.binned_statistic(rep_inds[hit_rows], np.vstack((ds, angles, weights)), statistic = angle_weighted_distance, bins=0.5 ) 

In [ ]:
num_pts = 30

eff_r = []
for num_pts in np.arange(1,101,1):

    wangle = np.pi/3

    ind = np.array([185300])

    start = (mesh.vertices-mesh.vertex_normals)[ind,:]
    starts = np.vstack(num_pts * [start])

    vs = oriented_vector_cone(num_points=num_pts,
                              center_vector=-mesh.vertex_normals[ind].reshape(3),
                              widest_angle=wangle)

    # lpa = trimesh_vtk.linked_point_actor(vertices_a=np.zeros((num_pts,3))+mesh.vertices[ind].reshape(3),
    #                                      vertices_b=1000 * vs + +mesh.vertices[ind].reshape(3), line_width=4, color=(1,0,0))



    # trimesh_vtk.render_actors([ma, lpa])

    rtrace = ray_inter.intersects_location(starts, vs, multiple_hits=False)

    orig_pt = mesh.vertices[ind]
    ds = np.linalg.norm(rtrace[0]-orig_pt, axis=1)

    angles = np.arccos(np.sum(mesh.face_normals[rtrace[2]] * vs[rtrace[1]], axis=1))

    med_angle = np.median(angles)
    std_angle = np.std(angles)
    good_rows = np.logical_and(angles>=med_angle-std_angle, angles<=med_angle+std_angle)

    weights = 1/angles

    eff_r.append(np.average(ds[good_rows], weights=weights[good_rows]))

In [ ]:
plt.plot(np.arange(1,101, 1), eff_r,'o')